# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.optim as optim
from torch.autograd import Variable


BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2), #padding=2保证输入输出尺寸相同
            nn.ReLU(),      #input_size=(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),#output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),      #input_size=(16*10*10)
            nn.MaxPool2d(2, 2)  #output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)

    # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # nn.Linear()的输入输出都是维度为1的值，所以要把多维度的tensor展平成一维
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3) 

Next, we can start to train and evaluate!

In [4]:
# train and evaluate
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #启用GPU
model = model.to(device)
for epoch in range(NUM_EPOCHS):
    train_correct=0
    train_total  =0
    for images, labels in tqdm(train_loader):
        images, labels= images.to(device), labels.to(device)
        images, labels = Variable(images), Variable(labels)  # 把数据转换成Variable
        optimizer.zero_grad()  # 优化器梯度初始化为零
        outputs = model(images)  # 把数据输入网络并得到输出，即进行前向传播
        loss = criterion(outputs,labels)  #交叉熵损失函数
        loss.backward()  # 反向传播梯度
        optimizer.step()  # 结束一次前传+反传之后，更新参数
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    with torch.no_grad():
        test_correct =0
        test_total   =0
        for images, labels in tqdm(test_loader):
            images, labels= images.to(device), labels.to(device)
            images, labels = Variable(images), Variable(labels)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    print('第{:d}个epoch的train识别准确率为：{:.2f}%'.format((epoch + 1),(100. * train_correct / train_total)))
    print('第{:d}个epoch的test识别准确率为：{:.2f}%'.format ((epoch + 1),(100. * test_correct / test_total)))


  1%|▋                                                                                 | 4/468 [00:00<00:14, 33.01it/s]

第1个epoch的train识别准确率为：89.98%
第1个epoch的test识别准确率为：97.51%


  1%|▌                                                                                 | 3/468 [00:00<00:16, 28.11it/s]

第2个epoch的train识别准确率为：97.49%
第2个epoch的test识别准确率为：98.11%


  1%|▋                                                                                 | 4/468 [00:00<00:14, 31.18it/s]

...

第8个epoch的train识别准确率为：99.33%
第8个epoch的test识别准确率为：98.69%


  1%|▋                                                                                 | 4/468 [00:00<00:15, 30.65it/s]

第9个epoch的train识别准确率为：99.42%
第9个epoch的test识别准确率为：98.87%


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 42.84it/s]

第10个epoch的train识别准确率为：99.47%
第10个epoch的test识别准确率为：98.72%


#### Q5:
Please print the training and testing accuracy.